In [1]:
from experiment import *
from cugal.config import SinkhornMethod, HungarianMethod, Config
import os
import torch

In [2]:
config = Config(
    device='cuda:1', 
    sinkhorn_method=SinkhornMethod.LOG,
    dtype=torch.float32,
    sinkhorn_threshold=1e-3,
    sinkhorn_iterations=500,
    mu=2,
    iter_count=15,
    use_sparse_adjacency=True,
    sinkhorn_cache_size=1,
    frank_wolfe_threshold=0,
    recompute_distance=True,
    hungarian_method=HungarianMethod.DENSE,
    )
experiment = Experiment(
    graphs=[
        #Graph(GraphKind.PREDEFINED_GRAPHS, {
        #    'source_file': 'data/MultiMagna/yeast0_Y2H1.txt',
        #    'target_file': f'data/MultiMagna/yeast{i}_Y2H1.txt',
        #}) for i in [25]#range(25, 6)#26, 5)
        #Graph(GraphKind.NEWMAN_WATTS, {'n': 100, 'k': 3, 'p': 0.05}),
        Graph(GraphKind.CA_HEP, {}),
    ],
    algorithms=[
        Algorithm(config, use_fugal=False),
        #Algorithm(config, use_fugal=True),
    ],
    noise_levels=[
        NoiseLevel(0.0, 0.0, False),
    ],
    save_alignment=True,
)
results = experiment.run()

print([result.accuracy for _, _, _, result in results.all_results()])

#folder = "results"
#if not os.path.exists(folder): os.makedirs(folder)
#results.dump(folder)

λ:   0%|          | 0/15 [00:00<?, ?it/s]

frank-wolfe:   0%|          | 0/10 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 17.61 GiB. GPU 1 has a total capacity of 23.69 GiB of which 5.64 GiB is free. Including non-PyTorch memory, this process has 17.92 GiB memory in use. Of the allocated memory 17.61 GiB is allocated by PyTorch, and 18.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 